In [1]:
# p6 notebook

In [2]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address      Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.0.4  104.37 KiB  16      68.8%             859e4ddf-49f8-4514-a14e-ae44731c9bca  rack1
UN  192.168.0.2  25.55 KiB   16      58.2%             3dc681e7-626d-45f8-9cc0-3726c0fef341  rack1
UN  192.168.0.3  104.38 KiB  16      73.0%             7dcb8420-5885-4764-befc-4a4ec9d82536  rack1



In [3]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [4]:
cass.execute("drop keyspace if exists weather")
cass.execute("create keyspace weather with replication={'class': 'SimpleStrategy', 'replication_factor': 3};")
cass.execute("use weather")
cass.execute("""
create type station_record (
    tmin INT,
    tmax INT
)
""")    
cass.execute("drop table if exists stations")
cass.execute("""
CREATE TABLE stations (
    id TEXT,
    date DATE,
    name TEXT STATIC,
    record weather.station_record,
    PRIMARY KEY ((id), date)
) WITH CLUSTERING ORDER BY (date ASC)
""")
    

In [5]:
#q1
cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [6]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-20dd3fca-fef0-4db2-aba2-5e9002f347b8;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [7]:
from pyspark.sql.functions import col, expr
df = spark.read.text("ghcnd-stations.txt")
df = df.withColumn("ID", expr("substring(value, 1, 11)"))
df = df.withColumn("STATE", expr("substring(value, 39, 2)"))
df = df.withColumn("NAME", expr("substring(value, 42, 30)"))
df = df.filter(col("state") == "WI")

for row in df.collect():
    name = row["NAME"].replace("'", "''")
    name = name.strip()
    cass.execute(f"""
    INSERT INTO weather.stations (id, name)
    VALUES ('{row["ID"]}', '{name}')
    """)

In [8]:
import pandas as pd
pd.DataFrame(cass.execute("SELECT COUNT(*) FROM weather.stations"))

,count
0,1313


In [9]:
#q2
pd.DataFrame(cass.execute("select id, name from weather.stations WHERE id = 'USW00014837'"))["name"][0]

'MADISON DANE CO RGNL AP'

In [10]:
#q3
q3 = pd.DataFrame(cass.execute("SELECT token(id) FROM weather.stations WHERE id = 'USC00470273'"))["system_token_id"][0]
q3

-9014250178872933741

In [11]:
#q4
import subprocess
import numpy

result = subprocess.check_output(
     "nodetool ring",
     stderr=subprocess.STDOUT,
     shell=True)
result = result.decode("utf-8")

vnodes = []
for line in result.splitlines():
    tokens = line.strip().split()
    if len(tokens) == 0:
        continue
    try:
        vnodes.append(numpy.int64(tokens[-1]))
    except:
        continue
        
vnodes = vnodes[1:]
first_node = None
for vnode in vnodes:
    if vnode > q3:
        first_node = vnode
        break

# wraps around
if first_node == None:
    first_node = vnode[0]

first_node

-8982054996226954036

In [12]:
from zipfile import ZipFile
from pyspark.sql import SparkSession
import os

parquet_df = []
try:
    with ZipFile('records.zip', 'r') as zip_ref:
        zip_ref.extractall()
    directory_path = 'records.parquet'

    for path in os.listdir(directory_path):
        file_path = os.path.join(directory_path, path)
        if path.strip().endswith('snappy.parquet'):
            df = spark.read.parquet(file_path)
            df = df.groupBy("station", "date").pivot("element", ["TMAX", "TMIN"]).sum("value")
            parquet_df.append(df)
except Exception as e:
    print(str(e))

datas = []
for df in parquet_df:
    for row in df.collect():
        station = str(row["station"])
        date = row["date"][:4] + "-" + row["date"][4:6] + "-" + row["date"][6:]
        tmin = int(row["TMIN"])
        tmax = int(row["TMAX"])
        datas.append([station, date, tmin, tmax])

In [13]:
import grpc
import station_pb2
import station_pb2_grpc

channel = grpc.insecure_channel("localhost:5440")
stub = station_pb2_grpc.StationStub(channel)

for data in datas:
    request = station_pb2.RecordTempsRequest(station = data[0], 
                                             date = data[1],
                                             tmin = data[2],
                                             tmax = data[3])
    response = stub.RecordTemps(request)
    if response.error != "":
        print(reponse.error)

In [14]:
#q5
request = station_pb2.StationMaxRequest(station = 'USW00014837')
response = stub.StationMax(request)
if response.error != "":
    print(response.error)
response.tmax

356

In [15]:
spark.catalog.dropTempView("stations")
stations_df = (spark.read.format("org.apache.spark.sql.cassandra")
.option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3")
.option("keyspace", "weather")
.option("table", "stations")
.load())
stations_df.createOrReplaceTempView("stations")

In [16]:
#q6
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [17]:
#q7
q7 = spark.sql("""
SELECT id, AVG(record.tmax - record.tmin) as diff
FROM stations
WHERE record IS NOT NULL
GROUP BY id
""")

output_dict = {}
for row in q7.collect():
    output_dict[row[0]] = row[1]
output_dict

{'USW00014898': 102.93698630136986,
 'USW00014837': 105.62739726027397,
 'USW00014839': 89.6986301369863,
 'USR0000WDDG': 102.06849315068493}

23/11/20 07:03:40 WARN ChannelPool: [s0|p6-db-2/192.168.0.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=7b79b3b6-2dc4-40bf-801a-2bb6f43b1b45, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700463681701}): failed to send request (java.nio.channels.NotYetConnectedException))
23/11/20 07:03:47 WARN ChannelPool: [s0|p6-db-2/192.168.0.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=7b79b3b6-2dc4-40bf-801a-2bb6f43b1b45, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700463681701}): failed to send request (java.nio.channels.NotYetConnectedException))
23/11/20 07:03:57 WARN ChannelPool: [s0|p6-db-

In [18]:
#q8
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address      Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.0.4  86.05 KiB  16      100.0%            859e4ddf-49f8-4514-a14e-ae44731c9bca  rack1
DN  192.168.0.2  86.04 KiB  16      100.0%            3dc681e7-626d-45f8-9cc0-3726c0fef341  rack1
UN  192.168.0.3  95.88 KiB  16      100.0%            7dcb8420-5885-4764-befc-4a4ec9d82536  rack1



In [19]:
#q9
request = station_pb2.StationMaxRequest(station = 'USW00014837')
response = stub.StationMax(request)
response.error

'need 3 replicas, but only have 2'

In [20]:
#q10
request = station_pb2.RecordTempsRequest(station = 'ABC00000001', 
                                             date = '2023-11-20',
                                             tmin = -100,
                                             tmax = 200)
response = stub.RecordTemps(request)
response.error

''

23/11/20 07:04:30 WARN ChannelPool: [s0|p6-db-2/192.168.0.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=7b79b3b6-2dc4-40bf-801a-2bb6f43b1b45, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700463681701}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
